In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from torchvision import transforms, models
from sklearn.metrics import classification_report
from tqdm import tqdm
from PIL import Image
import numpy as np
import os
import glob
from collections import Counter
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from pathlib import Path

BASE_DIR = Path.cwd().parent


In [ ]:
# ==========================================
# DATASET
# ==========================================

class ISICDataset(Dataset):
    def __init__(self, data_path, transforms=None, target_classes=None):
        self.data_path = data_path
        self.transforms = transforms
        self.target_classes = target_classes
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.target_classes)}

        self.image_paths = []
        self.labels = []

        for class_name in self.target_classes:
            class_path = os.path.join(data_path, class_name)

            paths = []
            for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG']:
                paths.extend(glob.glob(os.path.join(class_path, ext)))

            class_idx = self.class_to_idx[class_name]
            self.image_paths.extend(paths)
            self.labels.extend([class_idx] * len(paths))

    def __len__(self): return len(self.labels)
    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")

        if self.transforms:
            image = self.transforms(image)
        return image, self.labels[idx]

In [ ]:
# ==========================================
# MODEL
# ==========================================

# model update weight full
def get_transfer_model():
  model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)

  for layer in model.parameters():
    layer.requires_grad =False
  model.fc = nn.Sequential(
      nn.Dropout(p=0.4),
      nn.Linear(in_features=2048, out_features=1024),
      nn.LeakyReLU(),
      nn.Dropout(p=0.4),
      nn.Linear(in_features=1024, out_features=8),
  )

  return model

# freeze weight
def unfreeze_layers(model):
    for name, child in model.named_children():
        if name in ['layer3','layer4']:
            for layer in child.modules():
                if isinstance(layer, nn.BatchNorm2d):
                    layer.eval()
                    for param in layer.parameters():
                        param.requires_grad = False
                else:
                    for param in layer.parameters():
                        param.requires_grad = True

In [ ]:
# model = get_transfer_model()
# unfreeze_layers(model)
# for name, param in model.named_parameters():
#     print(name, param.shape, param.requires_grad)

In [ ]:
# model = get_transfer_model()
# for name, param in model.named_parameters():
#     print(name, param.shape, param.requires_grad)

In [ ]:
# folder
TRAIN_DIR = f"{BASE_DIR}/data/processed"
VALID_DIR = f"{BASE_DIR}/data/processed_valid"
TEST_DIR = f"{BASE_DIR}/data/processed_test"
SAVE_PATH = f"{BASE_DIR}/model\best_model.pt"

# class
TARGET_CLASSES = sorted([
    "AK",
    "BCC",
    "BKL",
    "DF",
    "MEL",
    "NV",
    "SCC",
    "VASC"
])

BATCH_SIZE = 16
EPOCHS = 150
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

IMG_SIZE = 224
stats = ((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

# dinh nghia transforms cho tap train va tap test
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(90),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.05),

    transforms.ToTensor(),
    transforms.Normalize(*stats),
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(*stats),
])


# Khoi tao data va dataloader
train_dataset = ISICDataset(TRAIN_DIR, transforms=train_transform, target_classes=TARGET_CLASSES)
valid_dataset = ISICDataset(VALID_DIR, transforms=test_transform, target_classes=TARGET_CLASSES)
test_dataset = ISICDataset(TEST_DIR, transforms=test_transform, target_classes=TARGET_CLASSES)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Tính Class Weights
class_counts = Counter(train_dataset.labels)
weights = [len(train_dataset) / (len(TARGET_CLASSES) * class_counts[i]) for i in range(len(TARGET_CLASSES))]
class_weights_tensor = torch.tensor(weights, dtype=torch.float).to(DEVICE)

# Model
model = get_transfer_model()
model = model.to(DEVICE)

# loss and optimize
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor, label_smoothing=0.1)

optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)


In [ ]:

writer = SummaryWriter('runs/isic_experiment')


# --- 3. Training Loop ---
best_loss = float('inf')

print("===== START =====")

for epoch in range(EPOCHS):
    if epoch == 8:
        print("\n ===== UNFREEZE LAYER 4 =====")
        unfreeze_layers(model)
        optimizer = optim.AdamW([
            {'params': model.layer3.parameters(), 'lr': 1e-5},
            {'params': model.layer4.parameters(), 'lr': 1e-5},
            {'params': model.fc.parameters(), 'lr': 1e-4}
        ], weight_decay=1e-4)
    # --- TRAINING PHASE ---
    model.train()
    train_loss = 0.0
    all_train_preds = []
    all_train_targets = []

    # Thanh progress bar cho train
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]", leave=False)

    for images, labels in train_bar:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        # Lưu dự đoán
        _, preds = torch.max(outputs, 1)
        all_train_preds.extend(preds.cpu().numpy())
        all_train_targets.extend(labels.cpu().numpy())

        train_bar.set_postfix(loss=loss.item())

    avg_train_loss = train_loss / len(train_loader)

    # --- VALIDATION ---
    model.eval()
    test_loss = 0.0
    all_val_preds = []
    all_val_targets = []

    valid_bar = tqdm(valid_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Val]", leave=False)

    with torch.no_grad():
        for images, labels in valid_bar:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, preds = torch.max(outputs, 1)

            all_val_preds.extend(preds.cpu().numpy())
            all_val_targets.extend(labels.cpu().numpy())

    avg_val_loss = test_loss / len(valid_loader)

    # tinh metrics
    precision = precision_score(all_val_targets, all_val_preds, average='weighted', zero_division=0)
    recall = recall_score(all_val_targets, all_val_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_val_targets, all_val_preds, average='weighted', zero_division=0)

    # --- Ghi Log TensorBoard ---
    writer.add_scalar('Loss/Train', avg_train_loss, epoch)
    writer.add_scalar('Loss/Val', avg_val_loss, epoch)
    writer.add_scalar('Metrics/Precision', precision, epoch)
    writer.add_scalar('Metrics/Recall', recall, epoch)
    writer.add_scalar('Metrics/F1', f1, epoch)

    # --- In kết quả ra màn hình ---
    print(f"\nEpoch [{epoch+1}/{EPOCHS}] Summary:")
    print(f"  Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
    print(f"  Precision:  {precision:.4f}   | Recall:    {recall:.4f}")

    # --- Classification Report (Mỗi 2 epoch) ---
    if (epoch + 1) % 2 == 0:
        print("\n--- Classification Report ---")
        print(classification_report(all_val_targets, all_val_preds, target_names=TARGET_CLASSES, zero_division=0))
        print("-----------------------------")

    # --- Lưu Model Tốt Nhất ---
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        torch.save(model.state_dict(), SAVE_PATH)
        print(f"  [SAVE] Đã lưu model tốt nhất (Loss giảm xuống {best_loss:.4f}) tại: {SAVE_PATH}")

print("\n XONG")
writer.close()

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        # Tính Cross Entropy Loss bình thường
        ce_loss = F.cross_entropy(inputs, targets, reduction='none', weight=self.alpha)

        # Tính pt (xác suất dự đoán đúng)
        pt = torch.exp(-ce_loss)

        # Công thức Focal Loss: (1 - pt)^gamma * log(pt)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [ ]:

# Tính Class Weights
class_counts = Counter(train_dataset.labels)
weights = [len(train_dataset) / (len(TARGET_CLASSES) * class_counts[i]) for i in range(len(TARGET_CLASSES))]
class_weights_tensor = torch.tensor(weights, dtype=torch.float).to(DEVICE)

# Model
checkpoint_path = f"{BASE_DIR}/model/best_model.pt"
model = get_transfer_model()
model = model.to(DEVICE)
checkpoint = torch.load(checkpoint_path, map_location=DEVICE)

try:
    if 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
    else:
        model.load_state_dict(checkpoint)
except Exception as e:
    print(f"loi {e}")

unfreeze_layers(model)

# loss and optimize
criterion = FocalLoss(alpha=class_weights_tensor, gamma=2.0)

optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-3)


In [ ]:

writer = SummaryWriter('runs/isic_experiment')
best_loss = float('inf')

print("===== START =====")

for epoch in range(EPOCHS):
    # --- TRAINING PHASE ---
    model.train()
    train_loss = 0.0
    all_train_preds = []
    all_train_targets = []

    # Thanh progress bar cho train
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]", leave=False)

    for images, labels in train_bar:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        # Lưu dự đoán
        _, preds = torch.max(outputs, 1)
        all_train_preds.extend(preds.cpu().numpy())
        all_train_targets.extend(labels.cpu().numpy())

        train_bar.set_postfix(loss=loss.item())

    avg_train_loss = train_loss / len(train_loader)

    # --- VALIDATION ---
    model.eval()
    test_loss = 0.0
    all_val_preds = []
    all_val_targets = []

    valid_bar = tqdm(valid_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Val]", leave=False)

    with torch.no_grad():
        for images, labels in valid_bar:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, preds = torch.max(outputs, 1)

            all_val_preds.extend(preds.cpu().numpy())
            all_val_targets.extend(labels.cpu().numpy())

    avg_val_loss = test_loss / len(valid_loader)

    # tinh metrics
    precision = precision_score(all_val_targets, all_val_preds, average='weighted', zero_division=0)
    recall = recall_score(all_val_targets, all_val_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_val_targets, all_val_preds, average='weighted', zero_division=0)

    # --- Ghi Log TensorBoard ---
    writer.add_scalar('Loss/Train', avg_train_loss, epoch)
    writer.add_scalar('Loss/Val', avg_val_loss, epoch)
    writer.add_scalar('Metrics/Precision', precision, epoch)
    writer.add_scalar('Metrics/Recall', recall, epoch)
    writer.add_scalar('Metrics/F1', f1, epoch)

    # --- In kết quả ra màn hình ---
    print(f"\nEpoch [{epoch+1}/{EPOCHS}] Summary:")
    print(f"  Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
    print(f"  Precision:  {precision:.4f}   | Recall:    {recall:.4f}")

    # --- Classification Report (Mỗi 2 epoch) ---
    if (epoch + 1) % 2 == 0:
        print("\n--- Classification Report ---")
        print(classification_report(all_val_targets, all_val_preds, target_names=TARGET_CLASSES, zero_division=0))
        print("-----------------------------")

    # --- Lưu Model Tốt Nhất ---
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        torch.save(model.state_dict(), SAVE_PATH)
        print(f"  [SAVE] Đã lưu model tốt nhất (Loss giảm xuống {best_loss:.4f}) tại: {SAVE_PATH}")

print("\n XONG")
writer.close()


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/runs/isic_experiment

In [ ]:
model.eval()
test_loss = 0.0
all_test_preds = []
all_test_targets = []

test_bar = tqdm(test_loader, desc=f"Epoch [Val]", leave=False)

with torch.no_grad():
     for images, labels in test_bar:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        loss = criterion(outputs, labels)

        test_loss += loss.item()
        _, preds = torch.max(outputs, 1)

        all_test_preds.extend(preds.cpu().numpy())
        all_test_targets.extend(labels.cpu().numpy())

avg_test_loss = test_loss / len(test_loader)

# tinh metrics
precision = precision_score(all_test_targets, all_test_preds, average='weighted', zero_division=0)
recall = recall_score(all_test_targets, all_test_preds, average='weighted', zero_division=0)
f1 = f1_score(all_test_targets, all_test_preds, average='weighted', zero_division=0)

    # --- In kết quả ra màn hình ---
print(f"Test Loss: {avg_test_loss:.4f}")
print(f"  Precision:  {precision:.4f}   | Recall:    {recall:.4f}")

    # --- Classification Report (Mỗi 2 epoch) ---
print("\n--- Classification Report ---")
print(classification_report(all_test_targets, all_test_preds, target_names=TARGET_CLASSES, zero_division=0))
print("-----------------------------")



Epoch [Val]:  45%|████▍     | 106/238 [11:11<12:24,  5.64s/it]